# Dependencies

In [8]:
!pip3 install -U -q pandas tldextract urllib3 scikit-learn


# Imports

In [9]:
import pandas as pd
from urllib.parse import urlparse
import tldextract
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

### Pre processing

In [16]:
data = pd.read_csv("malicious_phish.csv")
data["url"].duplicated().sum()

def is_valid_url(url):
    try:
        urlparse(url)
        return True
    except Exception:
        return False

def add_protocol(url):
    if not url.startswith(('http://', 'https://')):
        return 'http://' + url
    return url

def extract_domain_length(url):
    parsed_url = urlparse(url)
    return len(parsed_url.netloc)

def extract_domain_num(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc.count('.')

def extract_has_hyphen(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return int('-' in domain)

def extract_path_length(url):
    parsed_url = urlparse(url)
    return len(parsed_url.path)

def extract_num_path_segments(url):
    parsed_url = urlparse(url)
    return parsed_url.path.count('/')

def extract_tld_length(url):
    tld_info = tldextract.extract(url)
    return len(tld_info.suffix)

def extract_is_https(url):
    parsed_url = urlparse(url)
    return int(parsed_url.scheme == 'https')

def extract_num_query_params(url):
    parsed_url = urlparse(url)
    return len(parsed_url.query.split('&'))

def extract_has_redirect(url):
    return int('->' in url)

shortened_urls = ["bit.ly", "t.co", "ow.ly"]
def is_shortened(url):
    return int(any(short_url in url for short_url in shortened_urls))


data["url"] = data["url"].apply(add_protocol)
data = data[data["url"].apply(is_valid_url)].copy()

data["url_length"] = data["url"].apply(len)
data["domain_length"] = data["url"].apply(extract_domain_length)
data["domain_num"] = data["url"].apply(extract_domain_num)
data["has_hyphen"] = data["url"].apply(extract_has_hyphen)
data["path_length"] = data["url"].apply(extract_path_length)
data["num_path_segments"] = data["url"].apply(extract_num_path_segments)
data["tld_length"] = data["url"].apply(extract_tld_length)
data["is_https"] = data["url"].apply(extract_is_https)
data["num_query_params"] = data["url"].apply(extract_num_query_params)
data["has_redirect"] = data["url"].apply(extract_has_redirect)
data["is_shorted"] = data["url"].apply(is_shortened)

data.head()
data = data.drop(columns="url")

In [17]:
final = data
final["type"] = final["type"].replace({
    "benign": 0,
    "phishing": 1,
    "malware": 1,
    "defacement": 1
}).infer_objects(copy=False)
scaler = MinMaxScaler()
final = scaler.fit_transform(final)
final.head()

C:\Users\lyncon.baez\AppData\Local\Temp\ipykernel_761204\3465027115.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final["type"] = final["type"].replace({


AttributeError: 'numpy.ndarray' object has no attribute 'head'